In [8]:
import os
import json
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from networks.networks import *

In [9]:
with open('./rawdata/weibo3.json') as f:
    data = json.load(f)

In [10]:
model_root_path = './models/'
cls_tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-330M-Sentiment')
ner_tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-320M-Chinese', use_fast=False)
smy_tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Randeng-BART-139M-SUMMARY')

cls_model = torch.load(os.path.join(model_root_path, 'attitude_classify.model'))
ner_model = torch.load(os.path.join(model_root_path, 'named_entity_recognition.model'))
smy_model = torch.load(os.path.join(model_root_path, 'text_summary.model'))

In [ ]:
def static_item(item):
    

In [ ]:
for key in data.keys():
    for item in data[key]:
        